
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/freemocap/freemocap/blob/jon%2Fnpy_to_csv_ipynb/ipython_jupyter_notebooks/export_freemocap_npy_as_pandas_data_frame_csv.ipynb#scrollTo=xVLt0acPoAks]

# convert standard `freemocap` `npy` output to `pandas.DataFrame` `csv` files

## HOW TO USE
    change the `mediapipe_3d_npy_path` entry down there to the path to the `mediapipe_3d_whatever.npy` file from the session you want to convert
    this notebook will:
    - save out individual `npy` files for the `body`, `hands` and `face` data
        - 'tracked_points' aka 'landmarks' are in the 2nd dimension and follow the order listed here:
            - body - https://google.github.io/mediapipe/solutions/pose.html#pose-landmark-model-blazepose-ghum-3d
            - hands - https://google.github.io/mediapipe/solutions/hands.html#hand-landmark-model
            - face - need to do some work to get that order in any kind of useable state
    - save out individual `csv` files for the `body`, `hands` and `face` data (based on Pandas.DataFrame) with named columns showing the name of the `tracked_points`


- [ x ] load in `mediapipe_3d` data from a `freemocap` session
- [ x ]  get tracked_point names from `mediapipe`
- [ x ]  Create `pandas.DataFrame` objects for:
    - [ / ]  body_data
        - [ x ]  xyz data
        - [ ] reprojection_error data
    - [ x ]  right_hand_data
    - [ x ]  left_hand_data
    - [ / ]  face_data
        - [ x ] face mesh data (i.e. all data points)
        - [  ] face contour data (i.e. only the 'contour' parts of the face

In [26]:
from pathlib import Path
import pandas as pd
import numpy as np
from mediapipe.python.solutions import holistic as mp_holistic

In [ ]:
# Path to the `npy` file containing the `medipipe3d` data for the `freemocap` session you want to export
mediapipe_3d_npy_path = Path(r"C:\Users\jonma\freemocap_data\session_07-26-2022-08_07_30\output_data_files\mediapipe_3dData_numFrames_numTrackedPoints_spatialXYZ.npy") #path to the relevant `mediapipe_3d.npy` file


In [ ]:


mediapipe_3d_frame_trackedPoint_xyz = np.load(str(mediapipe_3d_npy_path))

print(f'loaded npy data with shape: {mediapipe_3d_frame_trackedPoint_xyz.shape}')


In [ ]:
pose_landmark_names = [landmark.name.lower() for landmark in mp_holistic.PoseLandmark]
hand_landmark_names = [landmark.name.lower() for landmark in mp_holistic.HandLandmark]
# face_landmark_names = [landmark.name.lower() for landmark in mp_holistic.PoseLandmark] #gonna have the clever for the face
print(f"Body tracked point names: {pose_landmark_names}")
print(hand_landmark_names)

In [ ]:
#get number of points in body, hands, face

number_of_body_points = len(pose_landmark_names)
number_of_hand_points = len(hand_landmark_names)

first_body_marker_index = 0
last_body_marker_index = number_of_body_points-1

first_right_hand_marker_index = last_body_marker_index + 1
last_right_hand_marker_index = number_of_body_points + number_of_hand_points-1

first_left_hand_marker_index = last_right_hand_marker_index + 1
last_left_hand_marker_index = last_right_hand_marker_index+1 + number_of_hand_points-1

first_face_marker_index = last_left_hand_marker_index + 1
last_face_marker_index = mediapipe_3d_frame_trackedPoint_xyz.shape[1]

number_of_face_points = last_face_marker_index - first_face_marker_index

print(f"body tracked point indices: {first_body_marker_index}:{last_body_marker_index}")
print(f"right hand tracked point indices: {first_right_hand_marker_index}:{last_right_hand_marker_index}")
print(f"left hand tracked point indices: {first_left_hand_marker_index}:{last_left_hand_marker_index}")
print(f"face tracked point indices: {first_face_marker_index}:{last_face_marker_index}")

print(f"number of body points: {last_body_marker_index-first_body_marker_index+1}")
print(f"number of right hand points: {last_right_hand_marker_index-first_right_hand_marker_index+1}")
print(f"number of left hand points: {last_left_hand_marker_index-first_left_hand_marker_index+1}")
print(f"number of face points: {last_face_marker_index-first_face_marker_index+1}")



In [ ]:
body_3d_xyz = mediapipe_3d_frame_trackedPoint_xyz[:,first_body_marker_index:last_body_marker_index+1,:]
right_hand_3d_xyz = mediapipe_3d_frame_trackedPoint_xyz[:,first_right_hand_marker_index:last_right_hand_marker_index+1,:]
left_hand_3d_xyz = mediapipe_3d_frame_trackedPoint_xyz[:,first_left_hand_marker_index:last_left_hand_marker_index+1,:]
face_3d_xyz = mediapipe_3d_frame_trackedPoint_xyz[:,first_face_marker_index:last_face_marker_index+1,:]

print(f"body 3d xyz shape: {body_3d_xyz.shape}")
print(f"right hand 3d xyz shape: {right_hand_3d_xyz.shape}")
print(f"left hand 3d xyz shape: {left_hand_3d_xyz.shape}")
print(f"face 3d xyz shape: {face_3d_xyz.shape}")

In [ ]:
# save broken up npy files
np.save(str(mediapipe_3d_npy_path.parent / "mediapipe_body_3d_xyz.npy"), body_3d_xyz)
np.save(str(mediapipe_3d_npy_path.parent / "mediapipe_right_hand_3d_xyz.npy"), right_hand_3d_xyz)
np.save(str(mediapipe_3d_npy_path.parent / "mediapipe_left_hand_3d_xyz.npy"), left_hand_3d_xyz)
np.save(str(mediapipe_3d_npy_path.parent / "mediapipe_face_3d_xyz.npy"), face_3d_xyz)

In [ ]:
# create pandas data frame headers

body_3d_xyz_header = []
for landmark_name in pose_landmark_names:
    body_3d_xyz_header.append(f"{landmark_name}_x")
    body_3d_xyz_header.append(f"{landmark_name}_y")
    body_3d_xyz_header.append(f"{landmark_name}_z")

right_hand_3d_xyz_header = []
for landmark_name in hand_landmark_names:
    right_hand_3d_xyz_header.append(f"right_hand_{landmark_name}_x")
    right_hand_3d_xyz_header.append(f"right_hand_{landmark_name}_y")
    right_hand_3d_xyz_header.append(f"right_hand_{landmark_name}_z")

left_hand_3d_xyz_header = []
for landmark_name in hand_landmark_names:
    left_hand_3d_xyz_header.append(f"left_hand_{landmark_name}_x")
    left_hand_3d_xyz_header.append(f"left_hand_{landmark_name}_y")
    left_hand_3d_xyz_header.append(f"left_hand_{landmark_name}_z")

face_3d_xyz_header = []
for landmark_number in range(last_face_marker_index - first_face_marker_index):
    face_3d_xyz_header.append(f"face_{str(landmark_number).zfill(4)}_x")
    face_3d_xyz_header.append(f"face_{str(landmark_number).zfill(4)}_y")
    face_3d_xyz_header.append(f"face_{str(landmark_number).zfill(4)}_z")

# print(f"body 3d xyz header: {body_3d_xyz_header}")
# print(f"right hand 3d xyz header: {right_hand_3d_xyz_header}")
# print(f"left hand 3d xyz header: {left_hand_3d_xyz_header}")
# print(f"face 3d xyz header: {face_3d_xyz_header}")

print(f"length of body 3d xyz header: {len(body_3d_xyz_header)}, should be: {number_of_body_points*3}")
print(f"length of right hand 3d xyz header: {len(right_hand_3d_xyz_header)}, should be: {number_of_hand_points*3}")
print(f"length of left hand 3d xyz header: {len(left_hand_3d_xyz_header)}, should be: {number_of_hand_points*3}")
print(f"length of face 3d xyz header: {len(face_3d_xyz_header)}, should be: {number_of_face_points*3}")

In [ ]:
number_of_frames = mediapipe_3d_frame_trackedPoint_xyz.shape[0]
body_flat = body_3d_xyz.reshape(number_of_frames, number_of_body_points*3)
body_flat.shape

body_dataframe = pd.DataFrame(body_flat, columns=body_3d_xyz_header)
body_dataframe.to_csv(str(mediapipe_3d_npy_path.parent / "mediapipe_body_3d_xyz.csv"), index=False)

right_hand_flat = right_hand_3d_xyz.reshape(number_of_frames, number_of_hand_points*3)
right_hand_dataframe = pd.DataFrame(right_hand_flat, columns=right_hand_3d_xyz_header)
right_hand_dataframe.to_csv(str(mediapipe_3d_npy_path.parent / "mediapipe_right_hand_3d_xyz.csv"), index=False)

left_hand_flat = left_hand_3d_xyz.reshape(number_of_frames, number_of_hand_points*3)
left_hand_dataframe = pd.DataFrame(left_hand_flat, columns=left_hand_3d_xyz_header)
left_hand_dataframe.to_csv(str(mediapipe_3d_npy_path.parent / "mediapipe_left_hand_3d_xyz.csv"), index=False)

face_flat = face_3d_xyz.reshape(number_of_frames, number_of_face_points*3)
face_dataframe = pd.DataFrame(face_flat, columns=face_3d_xyz_header)
face_dataframe.to_csv(str(mediapipe_3d_npy_path.parent / "mediapipe_face_3d_xyz.csv"), index=False)


In [27]:
pose_landmark_names = [landmark.name.lower() for landmark in mp_holistic.PoseLandmark]
hand_landmark_names = [landmark.name.lower() for landmark in mp_holistic.HandLandmark]
# face_landmark_names = [landmark.name.lower() for landmark in mp_holistic.PoseLandmark] #gonna have the clever for the face
print(f"Body tracked point names: {pose_landmark_names}")
print(hand_landmark_names)

Body tracked point names: ['nose', 'left_eye_inner', 'left_eye', 'left_eye_outer', 'right_eye_inner', 'right_eye', 'right_eye_outer', 'left_ear', 'right_ear', 'mouth_left', 'mouth_right', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_pinky', 'right_pinky', 'left_index', 'right_index', 'left_thumb', 'right_thumb', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 'left_heel', 'right_heel', 'left_foot_index', 'right_foot_index']
['wrist', 'thumb_cmc', 'thumb_mcp', 'thumb_ip', 'thumb_tip', 'index_finger_mcp', 'index_finger_pip', 'index_finger_dip', 'index_finger_tip', 'middle_finger_mcp', 'middle_finger_pip', 'middle_finger_dip', 'middle_finger_tip', 'ring_finger_mcp', 'ring_finger_pip', 'ring_finger_dip', 'ring_finger_tip', 'pinky_mcp', 'pinky_pip', 'pinky_dip', 'pinky_tip']


In [29]:
#get number of points in body, hands, face

number_of_body_points = len(pose_landmark_names)
number_of_hand_points = len(hand_landmark_names)

first_body_marker_index = 0
last_body_marker_index = number_of_body_points-1

first_right_hand_marker_index = last_body_marker_index + 1
last_right_hand_marker_index = number_of_body_points + number_of_hand_points-1

first_left_hand_marker_index = last_right_hand_marker_index + 1
last_left_hand_marker_index = last_right_hand_marker_index+1 + number_of_hand_points-1

first_face_marker_index = last_left_hand_marker_index + 1
last_face_marker_index = mediapipe_3d_frame_trackedPoint_xyz.shape[1]

number_of_face_points = last_face_marker_index - first_face_marker_index

print(f"body tracked point indices: {first_body_marker_index}:{last_body_marker_index}")
print(f"right hand tracked point indices: {first_right_hand_marker_index}:{last_right_hand_marker_index}")
print(f"left hand tracked point indices: {first_left_hand_marker_index}:{last_left_hand_marker_index}")
print(f"face tracked point indices: {first_face_marker_index}:{last_face_marker_index}")

print(f"number of body points: {last_body_marker_index-first_body_marker_index+1}")
print(f"number of right hand points: {last_right_hand_marker_index-first_right_hand_marker_index+1}")
print(f"number of left hand points: {last_left_hand_marker_index-first_left_hand_marker_index+1}")
print(f"number of face points: {last_face_marker_index-first_face_marker_index+1}")



body tracked point indices: 0:32
right hand tracked point indices: 33:53
left hand tracked point indices: 54:74
face tracked point indices: 75:553
number of body points: 33
number of right hand points: 21
number of left hand points: 21
number of face points: 479


In [30]:
body_3d_xyz = mediapipe_3d_frame_trackedPoint_xyz[:,first_body_marker_index:last_body_marker_index+1,:]
right_hand_3d_xyz = mediapipe_3d_frame_trackedPoint_xyz[:,first_right_hand_marker_index:last_right_hand_marker_index+1,:]
left_hand_3d_xyz = mediapipe_3d_frame_trackedPoint_xyz[:,first_left_hand_marker_index:last_left_hand_marker_index+1,:]
face_3d_xyz = mediapipe_3d_frame_trackedPoint_xyz[:,first_face_marker_index:last_face_marker_index+1,:]

print(f"body 3d xyz shape: {body_3d_xyz.shape}")
print(f"right hand 3d xyz shape: {right_hand_3d_xyz.shape}")
print(f"left hand 3d xyz shape: {left_hand_3d_xyz.shape}")
print(f"face 3d xyz shape: {face_3d_xyz.shape}")

body 3d xyz shape: (375, 33, 3)
right hand 3d xyz shape: (375, 21, 3)
left hand 3d xyz shape: (375, 21, 3)
face 3d xyz shape: (375, 478, 3)


In [31]:
# save broken up npy files
np.save(str(mediapipe_3d_npy_path.parent / "mediapipe_body_3d_xyz.npy"), body_3d_xyz)
np.save(str(mediapipe_3d_npy_path.parent / "mediapipe_right_hand_3d_xyz.npy"), right_hand_3d_xyz)
np.save(str(mediapipe_3d_npy_path.parent / "mediapipe_left_hand_3d_xyz.npy"), left_hand_3d_xyz)
np.save(str(mediapipe_3d_npy_path.parent / "mediapipe_face_3d_xyz.npy"), face_3d_xyz)

In [32]:
# create pandas data frame headers

body_3d_xyz_header = []
for landmark_name in pose_landmark_names:
    body_3d_xyz_header.append(f"{landmark_name}_x")
    body_3d_xyz_header.append(f"{landmark_name}_y")
    body_3d_xyz_header.append(f"{landmark_name}_z")

right_hand_3d_xyz_header = []
for landmark_name in hand_landmark_names:
    right_hand_3d_xyz_header.append(f"right_hand_{landmark_name}_x")
    right_hand_3d_xyz_header.append(f"right_hand_{landmark_name}_y")
    right_hand_3d_xyz_header.append(f"right_hand_{landmark_name}_z")

left_hand_3d_xyz_header = []
for landmark_name in hand_landmark_names:
    left_hand_3d_xyz_header.append(f"left_hand_{landmark_name}_x")
    left_hand_3d_xyz_header.append(f"left_hand_{landmark_name}_y")
    left_hand_3d_xyz_header.append(f"left_hand_{landmark_name}_z")

face_3d_xyz_header = []
for landmark_number in range(last_face_marker_index - first_face_marker_index):
    face_3d_xyz_header.append(f"face_{str(landmark_number).zfill(4)}_x")
    face_3d_xyz_header.append(f"face_{str(landmark_number).zfill(4)}_y")
    face_3d_xyz_header.append(f"face_{str(landmark_number).zfill(4)}_z")

# print(f"body 3d xyz header: {body_3d_xyz_header}")
# print(f"right hand 3d xyz header: {right_hand_3d_xyz_header}")
# print(f"left hand 3d xyz header: {left_hand_3d_xyz_header}")
# print(f"face 3d xyz header: {face_3d_xyz_header}")

print(f"length of body 3d xyz header: {len(body_3d_xyz_header)}, should be: {number_of_body_points*3}")
print(f"length of right hand 3d xyz header: {len(right_hand_3d_xyz_header)}, should be: {number_of_hand_points*3}")
print(f"length of left hand 3d xyz header: {len(left_hand_3d_xyz_header)}, should be: {number_of_hand_points*3}")
print(f"length of face 3d xyz header: {len(face_3d_xyz_header)}, should be: {number_of_face_points*3}")

length of body 3d xyz header: 99, should be: 99
length of right hand 3d xyz header: 63, should be: 63
length of left hand 3d xyz header: 63, should be: 63
length of face 3d xyz header: 1434, should be: 1434


In [33]:
number_of_frames = mediapipe_3d_frame_trackedPoint_xyz.shape[0]
body_flat = body_3d_xyz.reshape(number_of_frames, number_of_body_points*3)
body_flat.shape

body_dataframe = pd.DataFrame(body_flat, columns=body_3d_xyz_header)
body_dataframe.to_csv(str(mediapipe_3d_npy_path.parent / "mediapipe_body_3d_xyz.csv"), index=False)

right_hand_flat = right_hand_3d_xyz.reshape(number_of_frames, number_of_hand_points*3)
right_hand_dataframe = pd.DataFrame(right_hand_flat, columns=right_hand_3d_xyz_header)
right_hand_dataframe.to_csv(str(mediapipe_3d_npy_path.parent / "mediapipe_right_hand_3d_xyz.csv"), index=False)

left_hand_flat = left_hand_3d_xyz.reshape(number_of_frames, number_of_hand_points*3)
left_hand_dataframe = pd.DataFrame(left_hand_flat, columns=left_hand_3d_xyz_header)
left_hand_dataframe.to_csv(str(mediapipe_3d_npy_path.parent / "mediapipe_left_hand_3d_xyz.csv"), index=False)

face_flat = face_3d_xyz.reshape(number_of_frames, number_of_face_points*3)
face_dataframe = pd.DataFrame(face_flat, columns=face_3d_xyz_header)
face_dataframe.to_csv(str(mediapipe_3d_npy_path.parent / "mediapipe_face_3d_xyz.csv"), index=False)
